# Basic usage of Ctez and Checker

In this notebook, we're going to show how to compile, deploy and test the Checker library using Python and the [PyTezos](pytezos.org/) library.

We assume that the `checker` package was installed in the current environment with `python setup.py install`, that the LIGO compiler is available, as a command with the name `ligo`, and that a Tezos RPC node is available at the following url:

In [1]:
TEZOS_RPC="https://ghostnet.tezos.marigold.dev"



As this notebook is versioned, we recommend duplicating it before any modification.

Let's define a few helpers for PyTezos:

In [2]:
import pytezos as tz
import time
from checker_tools.client.operations import inject

alice = tz.Key.from_encoded_key("edsk3QoqBuvdamxouPhin7swCvkQNgq4jP5KZPbwWNnwdZpSpJiEbq")

In [3]:
alice.public_key_hash()

'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb'

In [4]:
ptz = tz.pytezos.using(TEZOS_RPC, alice)

In [5]:
ptz.balance()

Decimal('2000000.000000')

In [6]:
def do_it(call, amount=None):
    """Calls an entrypoint, optionally sending an amount of mutez"""
    if amount is not None:
            call = call.with_amount(amount)
    tx = call.as_transaction()
    inject(ptz, tx.autofill(ttl=None).sign())
    
def add_operator_fa2(owner, address, token, token_id):
    return do_it(token.update_operators([{
        "add_operator": {
            "owner": owner.public_key_hash(),
            "operator": address,
            "token_id": token_id
        }
    }]))

def add_allowance_fa12(token, spender, quantity):
    return do_it(token.approve({
        "spender": spender.address,
        "value": quantity
        })
    )

def now_plus_10():
    return int(time.time() + 10)

## Building and deploying

In [7]:
from checker_tools.client.compilation import compile_everything
from checker_tools.builder.config import CheckerRepo
from checker_tools.client import checker

Note that you probably won't be able to compile from a parent directory with a dockerized `ligo`.

In [8]:
conf = CheckerRepo("..")
conf.default_config


PosixPath('../checker.yaml')

In [ ]:
%time compile_everything(out_dir="generated/michelson", src_dir=conf.src, vendor_dir="../vendor")

Compilation of the project should take between 5 and 10 minutes.

Next, we can deploy our contracts, for later use.

In [10]:
ctez_contracts = checker.deploy_ctez(ptz, conf)
oracle = checker.deploy_contract(ptz, source_file="../utils/mock_oracle.tz",
                                 initial_storage={'owner': alice.public_key_hash(),
                                                  'price': (1, 1)})
cfmm = checker.deploy_contract(ptz, source_file="../utils/mock_cfmm_oracle.tz",
                               initial_storage={'owner': alice.public_key_hash(),
                                                'price': (1, 1)})
wtez = checker.deploy_wtez(ptz, conf)
wctez = checker.deploy_wctez(ptz,
                             conf,
                             ctez_fa12_address=ctez_contracts["fa12_ctez"].address)

Deploying ctez contract...
Done.
Deploying ctez FA1.2 contract...
Done.
Deploying ctez CFMM contract...
Done.
Deploying liquidity contract...
Done.
Setting liquidity address in CFMM contract...
Done.
Setting CFMM amd ctez FA1.2 addresses in ctez contract...
Done.


Loading config from ../checker.yaml


Deploying the wtez contract.


Loading config from ../checker.yaml


Done.
Tez Wrapper address: KT1HVYVoBZU2rxpdDqPKWo1FUMNUDx8KDVzn
Deploying the wctez contract.
Done.
wctez address: KT1MExecU9Bx6tyNbPDFawA9qEdida5r8ebz


And now, Checker. Checker is a big contract, made of several entrypoints that are independently deployed to a big map (sometimes in several chunks).

Initially, Checker is deployed in an "Unsealed" state; in this state, it isn't possible to use the contract, but it is possible to deploy the various entrypoints.

Once every entrypoint is deployed, it is possible to seal the contract.

In [11]:
ch = checker.deploy_checker(ptz, conf, oracle=oracle, collateral_fa2=wtez,
                       cfmm_token_fa2=wctez, ctez_cfmm=ctez_contracts["cfmm"])

Loading config from ../checker.yaml


Deploying the wrapper.
Checker address: KT1ENq2NgoV45XNAk937FYMsK2ymytuxvGM9
Deploying the TZIP-16 metadata.
Deploying TZIP-16 metadata: chunk 1 of 10
Deploying TZIP-16 metadata: chunk 2 of 10
Deploying TZIP-16 metadata: chunk 3 of 10
Deploying TZIP-16 metadata: chunk 4 of 10
Deploying TZIP-16 metadata: chunk 5 of 10
Deploying TZIP-16 metadata: chunk 6 of 10
Deploying TZIP-16 metadata: chunk 7 of 10
Deploying TZIP-16 metadata: chunk 8 of 10
Deploying TZIP-16 metadata: chunk 9 of 10
Deploying TZIP-16 metadata: chunk 10 of 10
Deploying: touch
  deployed: chunk 0.
  deployed: chunk 1.
  deployed: chunk 2.
  deployed: chunk 3.
  deployed: chunk 4.
Deploying: create_burrow
  deployed: chunk 0.
Deploying: deposit_collateral
  deployed: chunk 0.
Deploying: withdraw_collateral
  deployed: chunk 0.
Deploying: mint_kit
  deployed: chunk 0.
Deploying: burn_kit
  deployed: chunk 0.
Deploying: activate_burrow
  deployed: chunk 0.
Deploying: deactivate_burrow
  deployed: chunk 0.
Deploying: mark_for

In [12]:
ch.storage()

{'lazy_functions': 16,
 'metadata': 17,
 'deployment_state': {'sealed': {'burrows': 18,
   'cfmm': {'ctok': 1,
    'kit': 1,
    'kit_in_ctok_in_prev_block': {'den': 1, 'num': 1},
    'last_level': 600,
    'lqt': 1},
   'external_contracts': {'collateral_fa2': 'KT1HVYVoBZU2rxpdDqPKWo1FUMNUDx8KDVzn',
    'ctez_cfmm': 'KT1LLD2KhTtC7AjsfZDRLKACJM2NXcovvUNh',
    'ctok_fa2': 'KT1MExecU9Bx6tyNbPDFawA9qEdida5r8ebz',
    'oracle': 'KT1DEdWoaU6pRGzrYy47BQx6SoeUHVLTo133'},
   'fa2_state': {'ledger': 19, 'operators': 20},
   'last_ctez_in_tez': None,
   'last_index': None,
   'liquidation_auctions': {'avl_storage': {'last_ptr': 1, 'mem': 21},
    'burrow_slices': 22,
    'completed_auctions': None,
    'current_auction': None,
    'queued_slices': 1},
   'parameters': {'burrow_fee_index': 18446744073709551616,
    'circulating_kit': 0,
    'drift': 0,
    'drift_derivative': 0,
    'imbalance_index': 18446744073709551616,
    'index': 18446744073709551616,
    'last_touched': 1680150547,
    'o

PyTezos makes it easy to inspect the storage of our various smart contracts, which gives information such as the state of the CFMM or the current drift.

However, some information are stored in big maps, which are only shown here by a number (e.g., `lazy_functions` or FA2 `ledger`). We shall revisit them later in this tutorial.

## Minting some Ctez

TODO Checker CFMM 

We're going to create a Ctez oven, deposit some XTZ in it and mint some ctez. Let's call the corresponding entrypoint, `create`. Remember that you can get some documentation about the entrypoints names and arguments detected by PyTezos by doing `contract.entrypoint?` in Jupyter.

In [13]:
do_it(ctez_contracts["ctez"].create(
    {"id": 0, "delegate": None, "depositors":{"any": None}
}))

To get the oven address, we have to the `ovens` big map. In the future, we'll provide helpers with this in the Checker library.

In [14]:
ovens_id = ctez_contracts["ctez"].storage()["ovens"]
ovens = ptz.shell.head.context.big_maps[ovens_id]()
my_oven = ovens[0]["args"][0]["args"][0]["string"]
ovens

[{'prim': 'Pair',
  'args': [{'prim': 'Pair',
    'args': [{'string': 'KT1JRA6qvT4gwRVwCRQkcP3HrTsmjKXs8k5t'},
     {'int': '0'}]},
   {'int': '0'}]}]

In [15]:
my_oven

'KT1JRA6qvT4gwRVwCRQkcP3HrTsmjKXs8k5t'

In [16]:
my_oven = ptz.contract(my_oven)

In [17]:
do_it(my_oven.default(), amount=1000000000)
# Transfer 1G mutez = 1K tez to the oven

In [18]:
do_it(ctez_contracts["ctez"].mint_or_burn({"id":0, "quantity":int(1e8)}))
# Mint 100 Ctez

We can inspect the big map corresponding to Ctez ledger as such:

In [19]:
ctez_fa12_id = ctez_contracts["fa12_ctez"].storage()["tokens"]
ptz.shell.head.context.big_maps[ctez_fa12_id]()

[{'int': '100000000'}, {'int': '1'}]

Let's provide some liquidity to Ctez CFMM.

In [20]:
ctez_cfmm_storage = ctez_contracts["cfmm"].storage()
ctez_cfmm_storage

{'cashPool': 1,
 'tezPool': 1,
 'lqtTotal': 1,
 'target': 281474976710656,
 'ctez_address': 'KT1PekGXhfP1fUELN8BJRzJHs9BojT6w8xUf',
 'cashAddress': 'KT1PAuxJHfWNPEufcZsKERXGo6K8kJR98stv',
 'lqtAddress': 'KT1NkqCTzNEh4UUnP8e1VeZ7ftbGxqA2dxtj',
 'lastOracleUpdate': 1680150356,
 'consumerEntrypoint': 'KT1PekGXhfP1fUELN8BJRzJHs9BojT6w8xUf%cfmm_price'}

In [21]:
deposited = int(1e7)  # 10 ctez
lqt_minted = int(deposited * ctez_cfmm_storage["lqtTotal"] / ctez_cfmm_storage["cashPool"])

Don't forget to allow smart contracts to handle your tokens by doing:

In [22]:
add_allowance_fa12(ctez_contracts["fa12_ctez"],
                   ctez_contracts["cfmm"],
                   deposited)

In [23]:
do_it(ctez_contracts["cfmm"].addLiquidity({
    "owner": alice.public_key_hash(),
    "maxCashDeposited": deposited,   # Ctez deposited
    "minLqtMinted": lqt_minted-1,
    "deadline": int(time.time() + 15)
}), amount=deposited)   # XTZ deposited

Check that the CFMM has the correct liquidity:

In [24]:
ctez_contracts["cfmm"].storage()

{'cashPool': 10000001,
 'tezPool': 10000001,
 'lqtTotal': 10000001,
 'target': 281474976710656,
 'ctez_address': 'KT1PekGXhfP1fUELN8BJRzJHs9BojT6w8xUf',
 'cashAddress': 'KT1PAuxJHfWNPEufcZsKERXGo6K8kJR98stv',
 'lqtAddress': 'KT1NkqCTzNEh4UUnP8e1VeZ7ftbGxqA2dxtj',
 'lastOracleUpdate': 1680150356,
 'consumerEntrypoint': 'KT1PekGXhfP1fUELN8BJRzJHs9BojT6w8xUf%cfmm_price'}

## Using XTZ as collateral in Checker

Checker accepts a FA2 token as collateral. Depending the application you want to develop, it may be reasonable to use various tokens, including a new one. In the future, we will provide several examples.

The version of Checker we have deployed uses Wtez as collateral, a simple wrapper for XTZ:

In [25]:
wtez.address

'KT1HVYVoBZU2rxpdDqPKWo1FUMNUDx8KDVzn'

In [26]:
ch.storage()["deployment_state"]["sealed"]["external_contracts"]["collateral_fa2"]

'KT1HVYVoBZU2rxpdDqPKWo1FUMNUDx8KDVzn'

In [27]:
do_it(wtez.deposit(), amount=int(10e8))

We can then use the collateral to create a burrow — Checker's equivalent of an oven. Note that a minimum quantity of token for a deposit is defined at compile-time in Checker, in `src/constants.mligo`.

In [28]:
add_operator_fa2(alice, ch.address, wtez, 2)

In [29]:
do_it(ch.create_burrow((0, None, 1000000)))

We now have 1 burrow, in which we can deposit more wtez to mint some kits.

In [30]:
burrows_id = ch.storage()["deployment_state"]["sealed"]["burrows"]
ptz.shell.head.context.big_maps[burrows_id]()

[{'prim': 'Pair',
  'args': [{'prim': 'Pair',
    'args': [{'prim': 'Pair',
      'args': [{'prim': 'True'},
       {'string': 'KT1DVD59MBuUz5Xqs2ehwxrmvsP2VwyfV1zn'}]},
     {'int': '18446744073709551616'},
     {'int': '0'}]},
   {'prim': 'Pair',
    'args': [{'int': '0'}, {'string': '2023-03-30T04:29:07Z'}]},
   {'int': '0'}]}]

In [31]:
do_it(ch.deposit_collateral(0, int(10e7)))

Note that although we currently have 1kit = 1tez, Checker requires burrows to be overcollateralized by another compile-time constant, defined as `fminting` in `src/constants.mligo`.

By default, `fminting=210%`, which means that our burrow would have barely enough collateral if we mint the following quantity of kits:

In [32]:
do_it(ch.mint_kit(0, int((10/21)*10e7)))

In [33]:
kit_ledger_id = ch.storage()["deployment_state"]["sealed"]["fa2_state"]["ledger"]

In [34]:
ptz.shell.head.context.big_maps[kit_ledger_id]()

[{'int': '47619047'}]

TODO: 
 * selling kit
 * being liquidated